# Tugas Praktikum

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time

In [ ]:
# ---------------------------------------------------------------------------
# 1. Memuat dan Mempersiapkan Data
# ---------------------------------------------------------------------------
try:
    file_path = '/content/voice.csv'
    df = pd.read_csv(file_path)

    # Encoding label 'male'/'female' menjadi 1/0
    encoder = LabelEncoder()
    df['label'] = encoder.fit_transform(df['label']) # 0 = female, 1 = male

    # Pisahkan fitur (X) dan target (y)
    X = df.drop('label', axis=1)
    y = df['label']

    print(f"Data voice.csv dimuat. Bentuk: {X.shape}")

    # ---------------------------------------------------------------------------
    # 2. Definisi Skenario Uji Coba
    # ---------------------------------------------------------------------------
    # Kita akan menguji 2 rasio split dan 3 kernel
    split_ratios = [0.3, 0.2] # 0.3 -> 70:30, 0.2 -> 80:20
    kernels = ['linear', 'poly', 'rbf']

    # List untuk menyimpan hasil
    results = []

    # ---------------------------------------------------------------------------
    # 3. Looping Skenario
    # ---------------------------------------------------------------------------

    for test_size in split_ratios:
        split_name = f"{(1-test_size)*100:.0f}:{(test_size)*100:.0f}"
        print(f"\n--- Menguji Split {split_name} ---")

        # 1. Split Data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

        # 2. Feature Scaling (SANGAT PENTING untuk SVM)
        # Scaler di-fit HANYA pada data latih
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        # Data uji HANYA di-transform
        X_test_scaled = scaler.transform(X_test)

        # 3. Loop Kernel
        for kernel_name in kernels:
            start_time = time.time()
            print(f"Melatih SVM dengan kernel: {kernel_name}...")

            # 4. Inisialisasi dan Latih Model
            # Kita gunakan random_state untuk konsistensi
            model = SVC(kernel=kernel_name, random_state=42)
            model.fit(X_train_scaled, y_train)

            # 5. Prediksi dan Evaluasi
            y_pred = model.predict(X_test_scaled)
            accuracy = accuracy_score(y_test, y_pred)

            end_time = time.time()
            duration = end_time - start_time

            # 6. Simpan Hasil
            results.append({
                'Split Ratio': split_name,
                'Kernel': kernel_name,
                'Accuracy': accuracy,
                'Training Time (s)': duration
            })

            print(f"Selesai. Akurasi: {accuracy:.4f} (Waktu: {duration:.2f} d)")

    # ---------------------------------------------------------------------------
    # 4. Tabulasi Hasil
    # ---------------------------------------------------------------------------
    print("\n--- Tabulasi Performansi SVM ---")

    results_df = pd.DataFrame(results)

    # Mengatur 'Accuracy' untuk 4 desimal
    results_df['Accuracy'] = results_df['Accuracy'].map(lambda x: f"{x:.4f}")
    # Mengatur 'Training Time' untuk 2 desimal
    results_df['Training Time (s)'] = results_df['Training Time (s)'].map(lambda x: f"{x:.2f}")

    print(results_df.to_string(index=False))
    print("---------------------------------")


except FileNotFoundError:
    print(f"Error: File '{file_path}' tidak ditemukan.")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

Data voice.csv dimuat. Bentuk: (3168, 20)

--- Menguji Split 70:30 ---
Melatih SVM dengan kernel: linear...
Selesai. Akurasi: 0.9790 (Waktu: 0.08 d)
Melatih SVM dengan kernel: poly...
Selesai. Akurasi: 0.9600 (Waktu: 0.15 d)
Melatih SVM dengan kernel: rbf...
Selesai. Akurasi: 0.9832 (Waktu: 0.11 d)

--- Menguji Split 80:20 ---
Melatih SVM dengan kernel: linear...
Selesai. Akurasi: 0.9748 (Waktu: 0.08 d)
Melatih SVM dengan kernel: poly...
Selesai. Akurasi: 0.9558 (Waktu: 0.25 d)
Melatih SVM dengan kernel: rbf...
Selesai. Akurasi: 0.9826 (Waktu: 0.23 d)

--- Tabulasi Performansi SVM ---
Split Ratio Kernel Accuracy Training Time (s)
      70:30 linear   0.9790              0.08
      70:30   poly   0.9600              0.15
      70:30    rbf   0.9832              0.11
      80:20 linear   0.9748              0.08
      80:20   poly   0.9558              0.25
      80:20    rbf   0.9826              0.23
---------------------------------


Analisis Hasil:

Kernel Terbaik: Kernel RBF (Radial Basis Function) secara konsisten memberikan akurasi tertinggi pada kedua rasio split (98.32% pada 70:30 dan 98.26% pada 80:20). Ini menunjukkan bahwa data kemungkinan besar tidak dapat dipisahkan secara sempurna oleh garis lurus (linear) dan memerlukan batas keputusan yang lebih kompleks yang disediakan oleh RBF.

Performa Kernel Lain:

Kernel linear memiliki performa sangat baik (sekitar 97.5% - 97.9%), menunjukkan bahwa data ini sebagian besar linearly separable.

Kernel poly (polynomial) secara konsisten memberikan akurasi terendah (96.0% dan 95.6%) dan membutuhkan waktu latihan paling lama.

Pengaruh Split Ratio: Perbedaan antara split 70:30 dan 80:20 sangat kecil. Akurasi tertinggi ditemukan pada split 70:30 (data latih 70%), namun perbedaannya (0.0006) tidak signifikan secara statistik.

In [ ]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import time

In [ ]:
# --- 1. Definisi Fungsi Ekstraksi Fitur Histogram ---

def extract_histogram(image_path, bins=(8, 8, 8)):
    """
    Membaca gambar, mengubahnya ke HSV, menghitung histogram 3D (Hue, Saturation, Value),
    menormalisasi, dan meratakannya (flatten) menjadi vektor fitur 1D.
    """
    try:
        # Membaca gambar
        image = cv2.imread(image_path)
        if image is None:
            print(f"Peringatan: Tidak dapat membaca gambar {image_path}")
            return None

        # Konversi ke HSV (ruang warna yang lebih baik untuk histogram)
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Hitung histogram 3D.
        # Kita gunakan 'bins' (misal 8x8x8 = 512 fitur)
        hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])

        # Normalisasi histogram (penting untuk SVM)
        cv2.normalize(hist, hist)

        # 'Flatten' histogram menjadi 1D array
        return hist.flatten()
    except Exception as e:
        print(f"Error memproses {image_path}: {e}")
        return None

# --- 2. Definisi Fungsi untuk Memuat Data dari Folder ---

def load_data_from_dirs(base_dir):
    """
    Memuat semua gambar dari sub-folder 'siang' dan 'malam'
    di dalam direktori 'base_dir', lalu mengekstrak histogramnya.
    """
    features = []
    labels = []
    categories = ['day', 'night']

    for category in categories:
        path = os.path.join(base_dir, category)
        if not os.path.isdir(path):
            print(f"Error: Direktori tidak ditemukan: {path}")
            continue

        print(f"Memproses folder: {path}")
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)

            # Ekstrak histogram
            histogram = extract_histogram(img_path)

            if histogram is not None:
                features.append(histogram)
                labels.append(category)

    return np.array(features), np.array(labels)

# --- 3. Main Script ---

# Direktori berdasarkan struktur yang Anda berikan
train_dir = "/content/drive/MyDrive/Colab Notebooks/images/training"
test_dir = "/content/drive/MyDrive/Colab Notebooks/images/test"

# --- Memuat Data Latih dan Uji ---
print("Memulai memuat data latih...")
X_train, y_train = load_data_from_dirs(train_dir)

print("\nMemulai memuat data uji...")
X_test, y_test = load_data_from_dirs(test_dir)

if len(X_train) == 0 or len(X_test) == 0:
    print("\nError: Data latih atau data uji tidak berhasil dimuat.")
    print("Pastikan path direktori Anda (train_dir, test_dir) sudah benar dan berisi folder 'siang' dan 'malam'.")
else:
    print(f"\nData berhasil dimuat.")
    print(f"Total data latih: {len(X_train)} sampel")
    print(f"Total data uji: {len(X_test)} sampel")
    print(f"Total fitur per sampel (bins): {X_train.shape[1]}") # Akan jadi 512 jika bins=(8,8,8)

    # --- Encoding Label (String ke Angka) ---
    # Mengubah 'siang'/'malam' menjadi 0/1
    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.transform(y_test)

    # --- Hyperparameter Tuning (Eksperimen) ---
    print("\nMemulai Hyperparameter Tuning dengan GridSearchCV...")
    print("Ini mungkin memakan waktu beberapa menit...")

    # Parameter yang akan diuji untuk kernel RBF
    # C: Parameter regularisasi.
    # gamma: Koefisien kernel.
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001]
    }

    # Inisialisasi model SVM dengan kernel RBF
    svm = SVC(kernel='rbf')

    # Inisialisasi GridSearchCV
    # cv=5 berarti 5-fold cross-validation
    grid_search = GridSearchCV(svm, param_grid, cv=5, verbose=2, n_jobs=-1)

    start_tune = time.time()
    # Latih GridSearchCV pada data latih
    grid_search.fit(X_train, y_train)
    end_tune = time.time()

    print(f"\nTuning selesai dalam {end_tune - start_tune:.2f} detik.")

    # --- 4. Evaluasi Model ---

    # Tampilkan parameter terbaik
    print(f"\nParameter terbaik ditemukan: {grid_search.best_params_}")

    # Ambil model terbaik
    best_model = grid_search.best_estimator_

    # Lakukan prediksi pada data uji
    y_pred = best_model.predict(X_test)

    # Hitung akurasi
    accuracy = accuracy_score(y_test, y_pred)

    print("\n--- Hasil Performa pada Data Uji ---")
    print(f"Akurasi: {accuracy * 100:.2f}%")

    # Tampilkan laporan klasifikasi (precision, recall, f1-score)
    print("\nLaporan Klasifikasi:")
    print(classification_report(y_test, y_pred, target_names=encoder.classes_))

Memulai memuat data latih...
Memproses folder: /content/drive/MyDrive/Colab Notebooks/images/training/day
Memproses folder: /content/drive/MyDrive/Colab Notebooks/images/training/night

Memulai memuat data uji...
Memproses folder: /content/drive/MyDrive/Colab Notebooks/images/test/day
Memproses folder: /content/drive/MyDrive/Colab Notebooks/images/test/night

Data berhasil dimuat.
Total data latih: 240 sampel
Total data uji: 160 sampel
Total fitur per sampel (bins): 512

Memulai Hyperparameter Tuning dengan GridSearchCV...
Ini mungkin memakan waktu beberapa menit...
Fitting 5 folds for each of 16 candidates, totalling 80 fits

Tuning selesai dalam 3.73 detik.

Parameter terbaik ditemukan: {'C': 1, 'gamma': 1}

--- Hasil Performa pada Data Uji ---
Akurasi: 98.75%

Laporan Klasifikasi:
              precision    recall  f1-score   support

         day       0.99      0.99      0.99        80
       night       0.99      0.99      0.99        80

    accuracy                           0.